In [1]:

%pprint
import sys
if ('../py' not in sys.path): sys.path.insert(1, '../py')

Pretty printing has been turned OFF


In [2]:

from FRVRS import nu, fu
from pandas import read_excel, to_datetime, Series
import numpy as np

In [3]:

# load data frames
data_frames_list = nu.load_data_frames(metrics_evaluation_open_world_df='', first_responder_master_registry_file_stats_df='')
logs_df = data_frames_list['metrics_evaluation_open_world_df']
print(logs_df.shape) # (66069, 106)

Attempting to load /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/metrics_evaluation_open_world_df.pkl.
Attempting to load /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/first_responder_master_registry_file_stats_df.pkl.
(276926, 107)


In [4]:

mask_series = ~logs_df.patient_id.isnull()
sorted(logs_df[mask_series].patient_id.unique())

['Adept Shooter Root', 'Adept Victim Root', 'Broken Bob Root', 'Broken Gloria Root', 'Broken Helga Root', 'Civilian 1 Female Root', 'Civilian 1 Root', 'Civilian 2 Root', 'Intelligence Officer Root', 'Local Civilian with Internal Bleeding Root', 'Local Soldier 1 Root', 'Marine 1 Male Root', 'Marine 2 Male Root', 'Marine 3 Male Root', 'Marine 4 Male Root', 'Marine with Leg Amputation Root', 'Marine with Narrative Root', 'NPC 1 Root', 'NPC 2 Root', 'NPC 3 Root', 'NPC 4 Root', 'NPC Root', 'Navy Soldier 1 Male Root', 'Navy Soldier 2 Male Root', 'Navy Soldier 3 Male Root', 'Navy Soldier 4 Female Root', 'Navy Solider 4 Female Root', 'Open World Civilian 1 Male Root', 'Open World Civilian 2 Female Root', 'Open World Marine 1 Female Root', 'Open World Marine 1 Male Root', 'Open World Marine 2 Female Root', 'Open World Marine 2 Male Root', 'Open World Marine 3 Male Root', 'Open World Marine 4 Male Root', 'Patient U Root', 'Patient V Root', 'Patient W Root', 'Patient X Root', 'Simulation Root', '

In [5]:

df = data_frames_list['first_responder_master_registry_file_stats_df']
mask_series = ~df.responder_type.isnull()
sorted(df[mask_series].responder_type.unique())

['EM-FAC', 'EM-RES1', 'EM-RES2', 'EM-RES3', 'EMIM-RES1', 'EMIM-RES3', 'EMS FELO', 'EMT-Basic', 'EMT-I', 'EMT-Intermediate', 'M4-ATEM', 'M4-FM', 'M4-SURGERY', 'M4-SY', 'Nurse', 'Other HP', 'Paramedic', 'TEMS']

In [6]:

sorted([cn for cn in logs_df.columns if 'file' in cn])

[]


# File Stats Created for Metrics Evaluation Open World

In [7]:

if nu.pickle_exists('metrics_evaluation_open_world_file_stats_df'): file_stats_df = nu.load_object('metrics_evaluation_open_world_file_stats_df')
else:
    
    # Get the columns that consistently have only one value in them per session
    single_value_cols_set = set(logs_df.columns)
    for session_uuid, session_df in logs_df.groupby('session_uuid'):
        single_value_cols = set([col for col in session_df.columns if session_df[col].nunique() == 1])
        single_value_cols_set = single_value_cols_set.intersection(single_value_cols)
    
    # Get the learner types from the spreadsheet
    if nu.pickle_exists('disaster_day_itm_405_learner_types_df'): df = nu.load_object('disaster_day_itm_405_learner_types_df')
    else:
        file_path = '../data/xlsx/disaster_day_itm_405_3.6.2024.xlsx'
        df = read_excel(file_path)
        df.columns = [
            'session_file_date', 'session_file_name', 'session_uuid', 'responder_name', 'responder_type', 'site_name', 'encounter_layout'
        ]
        df = df.dropna(axis='columns', how='all')
        if 'session_file_date' in df.columns: df.session_file_date = to_datetime(df.session_file_date, infer_datetime_format=True)
        
        # Note the shape and save so you don't have to run it again
        print(df.shape) # (22, 6)
        nu.store_objects(disaster_day_itm_405_learner_types_df=df)
        nu.save_data_frames(disaster_day_itm_405_learner_types_df=df)
    
    # Merge the learner types from the spreadsheet in
    file_stats_df = logs_df[single_value_cols_set].dropna(axis='columns', how='all').merge(
        df, how='left', on='session_uuid'
    ).drop_duplicates().reset_index(drop=True)
    
    # Fill in any missing file dates
    for session_uuid, session_df in file_stats_df.groupby('session_uuid'):
        mask_series = session_df.session_file_date.isnull()
        if mask_series.any():
            session_file_date = fu.get_session_file_date(logs_df, session_uuid)
            file_stats_df.loc[session_df[mask_series].index, 'session_file_date'] = session_file_date
    
    # Save so you don't have to run it again
    nu.store_objects(metrics_evaluation_open_world_file_stats_df=file_stats_df)
    nu.save_data_frames(metrics_evaluation_open_world_file_stats_df=file_stats_df)
    
print(file_stats_df.shape) # (128, 18)

(86, 18)


In [8]:

# Remove any duplicate session IDs
mask_series = file_stats_df.duplicated(subset='session_uuid')
if mask_series.any():
    # display(file_stats_df[mask_series].dropna(axis='columns', how='all'))
    file_stats_df = file_stats_df[~mask_series]
    
    # Store the results and show the new data frame shape
    nu.store_objects(metrics_evaluation_open_world_file_stats_df=file_stats_df)
    nu.save_data_frames(metrics_evaluation_open_world_file_stats_df=file_stats_df)
    print(file_stats_df.shape) # (22, 16)

In [9]:

# Fix the null file dates
mask_series = file_stats_df.session_file_date.isnull()
if mask_series.any():
    print(f'I have {mask_series.sum()} scenes in my stats data frame without file dates.')
    for session_uuid, idx_df in file_stats_df[mask_series].groupby('session_uuid'):
        
        # Get the whole session history
        mask_series = (logs_df.session_uuid == session_uuid)
        session_df = logs_df[mask_series]
        
        session_file_date = session_df.event_time.min().date()
        file_stats_df.loc[idx_df.index, 'session_file_date'] = session_file_date
    nu.store_objects(metrics_evaluation_open_world_file_stats_df=file_stats_df)
    nu.save_data_frames(metrics_evaluation_open_world_file_stats_df=file_stats_df)
    mask_series = file_stats_df.session_file_date.isnull()
    print(f'I now have {mask_series.sum()} scenes in my stats data frame without file dates.')

I have 1 scenes in my stats data frame without file dates.
Pickling to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/metrics_evaluation_open_world_file_stats_df.pkl
Saving to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/metrics_evaluation_open_world_file_stats_df.csv
I now have 1 scenes in my stats data frame without file dates.


In [10]:

# Add file start time
if ('file_start_time' not in file_stats_df.columns) or ('file_stop_time' not in file_stats_df.columns):
    sub_directory = '../data/logs'
    import os.path as osp
    if 'file_name' in file_stats_df.columns:
        for file_name, idx_df in file_stats_df.groupby('file_name'):

            # Construct the full path to the file
            file_path = osp.join(sub_directory, file_name)

            # Attempt to read CSV file using pandas
            try:
                import pandas as pd
                file_df = pd.read_csv(file_path, header=None, index_col=False)

            # If unsuccessful, try using a reader
            except:
                rows_list = []
                with open(file_path, 'r') as f:
                    import csv
                    reader = csv.reader(f, delimiter=',', quotechar='"')
                    for values_list in reader:
                        if (values_list[-1] == ''): values_list.pop(-1)
                        rows_list.append({i: v for i, v in enumerate(values_list)})
                from pandas import DataFrame
                file_df = DataFrame(rows_list)

            ts_series = to_datetime(file_df[2], infer_datetime_format=True)
            file_stats_df.loc[idx_df.index, 'file_start_time'] = ts_series.min().to_pydatetime()
            file_stats_df.loc[idx_df.index, 'file_stop_time'] = ts_series.max().to_pydatetime()
        print(file_stats_df.shape) # (22, 18)
        nu.store_objects(metrics_evaluation_open_world_file_stats_df=file_stats_df)
        nu.save_data_frames(metrics_evaluation_open_world_file_stats_df=file_stats_df)

In [11]:

# Get a sample with a clear count of responders
new_column_name = 'is_a_one_triage_file'
# if (new_column_name in file_stats_df.columns): file_stats_df = file_stats_df.drop(columns=new_column_name)
if (new_column_name not in file_stats_df.columns):
    file_stats_df[new_column_name] = False
    data_frames_list = nu.load_data_frames(metrics_evaluation_open_world_scene_stats_df='')
    scene_stats_df = data_frames_list['metrics_evaluation_open_world_scene_stats_df']
    
    # Assume a 1:1 correspondence between file name and UUID from the logs data frame build
    for session_uuid, session_df in file_stats_df.groupby('session_uuid'):
        assert session_df.shape[0] == 1, "You've got duplicate session UUIDs"
        
        # Filter in the triage files in this UUID
        mask_series = (scene_stats_df.session_uuid == session_uuid) & (scene_stats_df.scene_type == 'Triage')
        
        # Get whether the file has only one triage run
        triage_scene_count = len(scene_stats_df[mask_series].groupby('scene_id').groups)
        is_a_one_triage_file = bool(triage_scene_count == 1)
        
        file_stats_df.loc[session_df.index, new_column_name] = is_a_one_triage_file
    
    # Store the results and show the new data frame shape
    nu.store_objects(metrics_evaluation_open_world_file_stats_df=file_stats_df)
    nu.save_data_frames(metrics_evaluation_open_world_file_stats_df=file_stats_df)
    
    print(file_stats_df.shape) # (22, 21)
display(file_stats_df.groupby(new_column_name).size().to_frame().rename(columns={0: 'record_count'}))

,record_count
is_a_one_triage_file,
False,86


In [12]:

new_column_name = 'is_in_registry'
if (new_column_name not in file_stats_df.columns):
    file_stats_df[new_column_name] = False
    
    # Store the results and show the new data frame shape
    nu.store_objects(metrics_evaluation_open_world_file_stats_df=file_stats_df)
    nu.save_data_frames(metrics_evaluation_open_world_file_stats_df=file_stats_df)
    
    print(file_stats_df.shape) # (22, 22)
display(file_stats_df.groupby(new_column_name).size().to_frame().rename(columns={0: 'record_count'}))

,record_count
is_in_registry,
False,86


In [35]:

# Add in all the json dats, if available
import json
import os
from os import path as osp

logs_path = '../data/logs/Metrics Evaluation Open World'
for sub_directory, directories_list, files_list in os.walk(logs_path):
    
    # Iterate over the files in the current subdirectory
    for file_name in files_list:
        
        # If the file is a JSON file, merge it into the subdirectory data frame
        if file_name.endswith('.json'):

            json_path = osp.join(sub_directory, file_name)
            with open(json_path, 'r') as f:
                file_json = json.load(f)
                flattened_json_dict = nu.get_row_dictionary(file_json, row_dict={}, key_prefix='')
                session_uuid = flattened_json_dict['sessionId']
                mask_series = (file_stats_df.session_uuid == session_uuid)
                for key, value in flattened_json_dict.items():
                    file_stats_df.loc[mask_series, key] = value

# Store the results and show the new data frame shape
nu.store_objects(metrics_evaluation_open_world_file_stats_df=file_stats_df)
nu.save_data_frames(metrics_evaluation_open_world_file_stats_df=file_stats_df)
print(file_stats_df.shape)

Pickling to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/metrics_evaluation_open_world_file_stats_df.pkl
Saving to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/metrics_evaluation_open_world_file_stats_df.csv
(86, 3971)


In [35]:

# Fix the encounter_layout column based on the set of patients in the scene
desert_patients_list = ['Open World Marine 1 Female Root', 'Open World Marine 2 Male Root', 'Open World Civilian 1 Male Root', 'Open World Civilian 2 Female Root']
jungle_patients_list = ['Open World Marine 1 Male Root', 'Open World Marine 2 Female Root', 'Open World Marine 3 Male Root', 'Open World Marine 4 Male Root']
submarine_patients_list = ['Navy Soldier 1 Male Root', 'Navy Soldier 2 Male Root', 'Navy Soldier 3 Male Root', 'Navy Soldier 4 Female Root']
urban_patients_list = ['Marine 1 Male Root', 'Marine 2 Male Root', 'Marine 3 Male Root', 'Marine 4 Male Root', 'Civilian 1 Female Root']
for (session_uuid, scene_id), scene_df in logs_df.groupby(fu.scene_groupby_columns):
    for env_str in ['desert', 'jungle', 'submarine', 'urban']:
        patients_list = eval(f'{env_str}_patients_list')
        if all(map(lambda patient_id: patient_id in scene_df.patient_id.unique().tolist(), patients_list)):
            mask_series = (file_stats_df.session_uuid == session_uuid)
            file_stats_df.loc[mask_series, 'encounter_layout'] = env_str.title()
            nu.store_objects(metrics_evaluation_open_world_file_stats_df=file_stats_df, verbose=False)
            nu.save_data_frames(metrics_evaluation_open_world_file_stats_df=file_stats_df, verbose=False)
display(file_stats_df.groupby('encounter_layout', dropna=False).size().to_frame().rename(columns={0: 'record_count'}))

,record_count
encounter_layout,
Desert,14
Discard,1
Jungle,14
Submarine,20
Urban,26
NaN,11



# Visualize

In [35]:

search_regex = re.compile('Urban', re.IGNORECASE)
columns_list = nu.get_regexed_columns(file_stats_df, search_regex=search_regex)
df = nu.get_regexed_dataframe(file_stats_df, columns_list, search_regex=search_regex)
df.sample(4)[columns_list].dropna(axis='columns', how='all').T

,51,180,77,167
encounter_layout,Urban,NaN,NaN,NaN
configData_scene,sim-urban,sim-urban-sanitized,sim-urban-sanitized,sim-urban-sanitized
configData_narrative,Urban Narrative 1,NaN,NaN,NaN
configData_scenarioData_name,Urban,Urban,Urban,Urban
configData_scenarioData_description,Urban Scenario,Urban Scenario,Urban Scenario,Urban Scenario
configData_narrative_narrativeSections00_additionalInfo,NaN,You are a medic in a civil affairs element ope...,You are a medic in a civil affairs element ope...,You are a medic in a civil affairs element ope...
configData_narrative_narrativeSections00_narratorRadioAudio_audioClip,NaN,Urban Narrative Audio/Urban Initialization,Urban Narrative Audio/Urban Initialization,Urban Narrative Audio/Urban Initialization
configData_narrative_narrativeSections01_narratorRadioAudio_audioClip,NaN,Urban Narrative Audio/Soartech/Urban Soartech ...,Urban Narrative Audio/Soartech/Urban Soartech ...,Urban Narrative Audio/Soartech/Urban Soartech ...
configData_narrative_narrativeSections02_narratorRadioAudio_audioClip,NaN,Urban Narrative Audio/Soartech/Soartech Radio 1,Urban Narrative Audio/Soartech/Soartech Radio 1,Urban Narrative Audio/Soartech/Soartech Radio 1
configData_narrative_narrativeSections03_narratorRadioAudio_audioClip,NaN,Urban Narrative Audio/Soartech/Soartech Radio 2,Urban Narrative Audio/Soartech/Soartech Radio 2,Urban Narrative Audio/Soartech/Soartech Radio 2


In [33]:

columns_list = ['encounter_layout', 'configData_narrative', 'configData_scene', 'configData_scenarioData_name', 'configData_scenarioData_description']
for cn in columns_list: display(file_stats_df.groupby(cn, dropna=False).size().to_frame().rename(columns={0: 'record_count'}))

,record_count
encounter_layout,
Discard,1
Submarine,9
Urban,5
NaN,71


,record_count
configData_narrative,
Base Scene Narrative 1,2
Desert Narrative 1,1
Submarine Narrative 1,4
Urban Narrative 1,4
NaN,75


,record_count
configData_scene,
sim-desert,14
sim-jungle,15
sim-none,2
sim-sub,17
sim-urban,4
sim-urban-sanitized,20
NaN,14


,record_count
configData_scenarioData_name,
Desert,11
ITM Tutorial,2
Jungle,3
Jungle Eval,15
Submarine,4
Submarine Eval,13
Urban,24
NaN,14


,record_count
configData_scenarioData_description,
Desert,11
Evaluation Jungle Scenario with Six Casualties and One Narrative Delivering Marine,3
ITM Tutorial in Default Room,2
Jungle Eval,15
Submarine Scenario,4
Urban Scenario,24
submarine Eval Scenario,13
NaN,14


In [38]:

import itertools

# Print the pairs
for pair in itertools.combinations(columns_list, 2): display(file_stats_df[list(pair)].drop_duplicates().sort_values(list(pair)))

,encounter_layout,configData_narrative
66,Discard,Desert Narrative 1
36,Submarine,Base Scene Narrative 1
30,Submarine,Submarine Narrative 1
69,Submarine,Urban Narrative 1
33,Submarine,NaN
45,Urban,Urban Narrative 1
27,Urban,NaN
0,NaN,NaN


,encounter_layout,configData_scene
66,Discard,sim-desert
36,Submarine,sim-none
30,Submarine,sim-sub
69,Submarine,sim-urban
33,Submarine,NaN
45,Urban,sim-urban
27,Urban,NaN
12,NaN,sim-desert
6,NaN,sim-jungle
15,NaN,sim-sub


,encounter_layout,configData_scenarioData_name
66,Discard,Jungle
36,Submarine,ITM Tutorial
30,Submarine,Submarine
69,Submarine,Urban
33,Submarine,NaN
45,Urban,Urban
27,Urban,NaN
12,NaN,Desert
183,NaN,Jungle
6,NaN,Jungle Eval


,encounter_layout,configData_scenarioData_description
66,Discard,Evaluation Jungle Scenario with Six Casualties...
36,Submarine,ITM Tutorial in Default Room
30,Submarine,Submarine Scenario
69,Submarine,Urban Scenario
33,Submarine,NaN
45,Urban,Urban Scenario
27,Urban,NaN
12,NaN,Desert
183,NaN,Evaluation Jungle Scenario with Six Casualties...
6,NaN,Jungle Eval


,configData_narrative,configData_scene
36,Base Scene Narrative 1,sim-none
66,Desert Narrative 1,sim-desert
30,Submarine Narrative 1,sim-sub
45,Urban Narrative 1,sim-urban
12,NaN,sim-desert
6,NaN,sim-jungle
15,NaN,sim-sub
0,NaN,sim-urban-sanitized
27,NaN,NaN


,configData_narrative,configData_scenarioData_name
36,Base Scene Narrative 1,ITM Tutorial
66,Desert Narrative 1,Jungle
30,Submarine Narrative 1,Submarine
45,Urban Narrative 1,Urban
12,NaN,Desert
183,NaN,Jungle
6,NaN,Jungle Eval
15,NaN,Submarine Eval
0,NaN,Urban
27,NaN,NaN


,configData_narrative,configData_scenarioData_description
36,Base Scene Narrative 1,ITM Tutorial in Default Room
66,Desert Narrative 1,Evaluation Jungle Scenario with Six Casualties...
30,Submarine Narrative 1,Submarine Scenario
45,Urban Narrative 1,Urban Scenario
12,NaN,Desert
183,NaN,Evaluation Jungle Scenario with Six Casualties...
6,NaN,Jungle Eval
0,NaN,Urban Scenario
15,NaN,submarine Eval Scenario
27,NaN,NaN


,configData_scene,configData_scenarioData_name
12,sim-desert,Desert
66,sim-desert,Jungle
6,sim-jungle,Jungle Eval
36,sim-none,ITM Tutorial
30,sim-sub,Submarine
15,sim-sub,Submarine Eval
45,sim-urban,Urban
0,sim-urban-sanitized,Urban
27,NaN,NaN


,configData_scene,configData_scenarioData_description
12,sim-desert,Desert
66,sim-desert,Evaluation Jungle Scenario with Six Casualties...
6,sim-jungle,Jungle Eval
36,sim-none,ITM Tutorial in Default Room
30,sim-sub,Submarine Scenario
15,sim-sub,submarine Eval Scenario
45,sim-urban,Urban Scenario
0,sim-urban-sanitized,Urban Scenario
27,NaN,NaN


,configData_scenarioData_name,configData_scenarioData_description
12,Desert,Desert
36,ITM Tutorial,ITM Tutorial in Default Room
66,Jungle,Evaluation Jungle Scenario with Six Casualties...
6,Jungle Eval,Jungle Eval
30,Submarine,Submarine Scenario
15,Submarine Eval,submarine Eval Scenario
0,Urban,Urban Scenario
27,NaN,NaN


In [40]:

columns_list = ['configData_scene', 'configData_scenarioData_name', 'configData_scenarioData_description']
display(file_stats_df[columns_list].drop_duplicates().sort_values(columns_list))
display(file_stats_df.groupby(columns_list, dropna=False).size().to_frame().rename(columns={0: 'record_count'}))

,configData_scene,configData_scenarioData_name,configData_scenarioData_description
12,sim-desert,Desert,Desert
66,sim-desert,Jungle,Evaluation Jungle Scenario with Six Casualties...
6,sim-jungle,Jungle Eval,Jungle Eval
36,sim-none,ITM Tutorial,ITM Tutorial in Default Room
30,sim-sub,Submarine,Submarine Scenario
15,sim-sub,Submarine Eval,submarine Eval Scenario
45,sim-urban,Urban,Urban Scenario
0,sim-urban-sanitized,Urban,Urban Scenario
27,NaN,NaN,NaN


record_count
configData_scene    configData_scenarioData_name configData_scenarioData_description                             
sim-desert          Desert                       Desert                                                        11
                    Jungle                       Evaluation Jungle Scenario with Six Casualties ...             3
sim-jungle          Jungle Eval                  Jungle Eval                                                   15
sim-none            ITM Tutorial                 ITM Tutorial in Default Room                                   2
sim-sub             Submarine                    Submarine Scenario                                             4
                    Submarine Eval               submarine Eval Scenario                                       13
sim-urban           Urban                        Urban Scenario                                                 4
sim-urban-sanitized Urban                        Urban Scenario                                                20
NaN                 NaN                          NaN                                                           14

In [14]:

file_stats_df.sample(5).T

,161,54,80,222,192
session_uuid,c6d3a90f-68c0-4948-bd96-537e80973605,8cb0ca8f-b626-4bc7-bfd5-1f44770e088a,385032e9-9801-4dcf-a841-b3703a0d9acd,92c78261-0f01-4d4f-8ae3-7aa875d6cd7b,287389c4-4c48-4483-87c0-6b363b57bde2
logger_version,1.4,1.4,1.4,1.4,1.4
player_location_left_hand_location,"(0.0, 0.0, 0.0)","(0.0, 0.0, 0.0)",NaN,"(0.0, 0.0, 0.0)","(0.0, 0.0, 0.0)"
patient_demoted_health_level,NaN,NaN,NaN,NaN,NaN
file_name,Metrics Evaluation Open World/ITM 3.20.2024 40...,Metrics Evaluation Open World/ITM 3.13.2024/8c...,Metrics Evaluation Open World/ITM 3.14.2024 40...,Metrics Evaluation Open World/ITM 3.20.2024 40...,Metrics Evaluation Open World/ITM 3.20.2024 40...
player_location_right_hand_location,"(0.0, 0.0, 0.0)","(0.0, 0.0, 0.0)",NaN,"(0.0, 0.0, 0.0)","(0.0, 0.0, 0.0)"
patient_demoted_health_time_remaining,NaN,NaN,NaN,NaN,NaN
session_file_date,2024-03-20 00:00:00,2024-03-06 00:00:00,2024-03-14 00:00:00,2024-03-20 00:00:00,2024-03-20 00:00:00
session_file_name,NaN,8cb0ca8f-b626-4bc7-bfd5-1f44770e088a.csv,NaN,NaN,NaN
responder_type,NaN,EM-RES1,NaN,NaN,NaN


In [15]:

# Get the unique count of each column
for cn in sorted(file_stats_df.columns): print(cn, file_stats_df[cn].nunique())

encounter_layout 3
file_name 85
file_start_time 84
file_stop_time 85
is_a_one_triage_file 1
is_in_registry 1
logger_version 1
patient_demoted_health_level 1
patient_demoted_health_time_remaining 1
player_location_left_hand_location 1
player_location_right_hand_location 1
responder_type 1
session_file_date 6
session_file_name 15
session_uuid 85
site_name 1


In [16]:

sorted(file_stats_df.columns)

['encounter_layout', 'file_name', 'file_start_time', 'file_stop_time', 'is_a_one_triage_file', 'is_in_registry', 'logger_version', 'patient_demoted_health_level', 'patient_demoted_health_time_remaining', 'player_location_left_hand_location', 'player_location_right_hand_location', 'responder_type', 'session_file_date', 'session_file_name', 'session_uuid', 'site_name']

In [17]:

display(file_stats_df.groupby('responder_type').size().to_frame().rename(columns={0: 'record_count'}).sort_values(
    'record_count', ascending=False
).head(5))

,record_count
responder_type,
EM-RES1,15
